In [0]:
%pip install -U -qqqq mlflow databricks-langchain databricks-agents uv langgraph==0.3.4 langchain_mcp_adapters
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%%writefile agent.py
# from typing import Any, Generator, Optional, Sequence, Union

# # get_tools is not needed here anymore as we call the client directly
# # from get_tools import get_nimble_tools
# import asyncio
# import mlflow
# from databricks_langchain import (
#     ChatDatabricks,
#     UCFunctionToolkit,
#     VectorSearchRetrieverTool,
# )
# from langchain_core.language_models import LanguageModelLike
# from langchain_core.runnables import RunnableConfig, RunnableLambda
# from langchain_core.tools import BaseTool
# from langgraph.graph import END, StateGraph
# from langgraph.graph.graph import CompiledGraph
# from langgraph.graph.state import CompiledStateGraph
# from langgraph.prebuilt.tool_node import ToolNode
# from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
# from mlflow.pyfunc import ChatAgent
# from mlflow.types.agent import (
#     ChatAgentChunk,
#     ChatAgentMessage,
#     ChatAgentResponse,
#     ChatContext,
# )

# from langchain_mcp_adapters.client import MultiServerMCPClient

# # NOTE: The creation of the agent and its dependencies are now inside
# # the async create_agent() function.

# def create_tool_calling_agent(
#     model: LanguageModelLike,
#     tools: Union[ToolNode, Sequence[BaseTool]],
#     system_prompt: Optional[str] = None,
# ) -> CompiledGraph:
#     model = model.bind_tools(tools)

#     def should_continue(state: ChatAgentState):
#         messages = state["messages"]
#         last_message = messages[-1]
#         if last_message.get("tool_calls"):
#             return "continue"
#         else:
#             return "end"

#     if system_prompt:
#         preprocessor = RunnableLambda(
#             lambda state: [{"role": "system", "content": system_prompt}]
#             + state["messages"]
#         )
#     else:
#         preprocessor = RunnableLambda(lambda state: state["messages"])
#     model_runnable = preprocessor | model

#     def call_model(
#         state: ChatAgentState,
#         config: RunnableConfig,
#     ):
#         response = model_runnable.invoke(state, config)
#         return {"messages": [response]}

#     workflow = StateGraph(ChatAgentState)
#     workflow.add_node("agent", RunnableLambda(call_model))
#     workflow.add_node("tools", ChatAgentToolNode(tools))
#     workflow.set_entry_point("agent")
#     workflow.add_conditional_edges(
#         "agent", should_continue, {"continue": "tools", "end": END}
#     )
#     workflow.add_edge("tools", "agent")
#     return workflow.compile()


# class LangGraphChatAgent(ChatAgent):
#     def __init__(self, agent: CompiledStateGraph):
#         self.agent = agent

#     # This remains a synchronous function
#     # def predict(
#     #     self,
#     #     messages: list[ChatAgentMessage],
#     #     context: Optional[ChatContext] = None,
#     #     custom_inputs: Optional[dict[str, Any]] = None,
#     # ) -> ChatAgentResponse:
#     #     request = {"messages": self._convert_messages_to_dict(messages)}
#     #     response_messages = []
#     #     for event in self.agent.stream(request, stream_mode="updates"):
#     #         for node_data in event.values():
#     #             response_messages.extend(
#     #                 ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
#     #             )
#     #     return ChatAgentResponse(messages=response_messages)

#     def predict(
#         self,
#         messages: list[ChatAgentMessage],
#         context: Optional[ChatContext] = None,
#         custom_inputs: Optional[dict[str, Any]] = None,
#     ) -> ChatAgentResponse:
#         request = {"messages": self._convert_messages_to_dict(messages)}
#         response_messages = []
#         for event in self.agent.astream(request, stream_mode="updates"):
#             for node_data in event.values():
#                 # Check if messages are already ChatAgentMessage instances
#                 # If they are, just extend the list directly
#                 # Otherwise, assume they are dictionaries and create ChatAgentMessage objects
#                 if node_data.get("messages") and isinstance(node_data["messages"][0], ChatAgentMessage):
#                     response_messages.extend(node_data["messages"])
#                 else:
#                     response_messages.extend(
#                         ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
#                     )
#         return ChatAgentResponse(messages=response_messages)

#     def predict_stream(
#         self,
#         messages: list[ChatAgentMessage],
#         context: Optional[ChatContext] = None,
#         custom_inputs: Optional[dict[str, Any]] = None,
#     ) -> Generator[ChatAgentChunk, None, None]:
#         request = {"messages": self._convert_messages_to_dict(messages)}
#         for event in self.agent.astream(request, stream_mode="updates"):
#             for node_data in event.values():
#                 yield from (
#                     ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
#                 )

# # NEW ASYNC INITIALIZATION FUNCTION
# async def create_agent():
#     """
#     An asynchronous function to initialize and return the agent.
#     This safely handles the async call to get tools.
#     """
#     # LLM and system prompt setup
#     # llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")
#     # llm = ChatDatabricks(endpoint="databricks-claude-3-7-sonnet")
#     llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")


    
#     system_prompt = ""

#     # Initialize tools list
#     tools = []
#     uc_tool_names = ["system.ai.python_exec"]
#     uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
#     tools.extend(uc_toolkit.tools)

#     # Nimble MCP Tools setup
#     nimble_client = MultiServerMCPClient({
#         "nimble": {
#             "url": "https://mcp.nimbleway.com/sse",
#             "transport": "sse",
#             "headers": {
#                 "Authorization": f"Bearer 1c7cdd787c154fe3a78ebcc3428d82c48f4fada38e0042c6a427fbe7e3b842ae"
#             }
#         }
#     })
    
#     # *** THIS IS THE KEY CHANGE ***
#     # We now 'await' this call inside our async function instead of using asyncio.run()
#     nimble_tools = await nimble_client.get_tools()
#     tools.extend(nimble_tools)

#     # Create and compile the agent graph
#     compiled_agent_graph = create_tool_calling_agent(llm, tools, system_prompt)
    
#     # Instantiate our final agent object
#     agent = LangGraphChatAgent(compiled_agent_graph)
    
#     # Set the model for MLflow (optional, but good practice)
#     # mlflow.models.set_model(agent)

#     return agent

# # We NO LONGER create a global AGENT instance here.
# # AGENT = ...


# %%writefile agent.py
from typing import Any, Generator, Optional, Sequence, Union

import uuid # Add this import for generating IDs if needed


import asyncio
import mlflow
import json
from databricks_langchain import (
    ChatDatabricks,
    UCFunctionToolkit,
    VectorSearchRetrieverTool,
)
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode

# Import specific message types from langchain_core.messages
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, BaseMessage, SystemMessage

from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ( # <-- Ensure these are imported
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
# Add these specific imports for MLflow's tool call types
# from mlflow.types.chat import ToolCall, FunctionCall

from langchain_mcp_adapters.client import MultiServerMCPClient


# --- Helper function to convert LangChain BaseMessage to MLflow ChatAgentMessage ---
def _to_mlflow_chat_agent_message(lc_message: BaseMessage) -> ChatAgentMessage:
    if isinstance(lc_message, HumanMessage):
        return ChatAgentMessage(role="user", content=lc_message.content, id=str(uuid.uuid4()))
    elif isinstance(lc_message, AIMessage):
        mlflow_tool_calls = []
        if lc_message.tool_calls:
            for lc_tc in lc_message.tool_calls:
                # CRITICAL FIX: Construct the dictionary structure expected by MLflow's ChatAgentMessage
                function_name = lc_tc.get("name")
                function_arguments_str = json.dumps(lc_tc.get("args")) 
                tool_call_id = lc_tc.get("id", str(uuid.uuid4())) # Ensure ID exists

                # Construct the dictionary for the 'function' part
                function_dict = {
                    "name": function_name,
                    "arguments": function_arguments_str
                }
                # Construct the dictionary for the full 'tool_call'
                tool_call_dict = {
                    "id": tool_call_id,
                    "function": function_dict # Nested dictionary
                }
                mlflow_tool_calls.append(tool_call_dict) # Append the dictionary
                
        # Ensure content is not None, replace with empty string if it is
        content_to_use = lc_message.content if lc_message.content is not None else ""

        return ChatAgentMessage(
            role="assistant", 
            content=content_to_use, # Use the potentially cleaned content
            tool_calls=mlflow_tool_calls,
            id=str(uuid.uuid4()) # ID for the ChatAgentMessage itself
        )
    elif isinstance(lc_message, ToolMessage):
        return ChatAgentMessage(
            role="tool", 
            content=lc_message.content, 
            name=lc_message.name, 
            tool_call_id=lc_message.tool_call_id, # Use the ID from the LangChain ToolMessage
            id=str(uuid.uuid4()) # ID for the ChatAgentMessage itself
        )
    elif isinstance(lc_message, SystemMessage):
        return ChatAgentMessage(role="system", content=lc_message.content, id=str(uuid.uuid4()))
    else:
        # Fallback for any other BaseMessage type
        # Ensure content is not None, replace with empty string if it is
        content_to_use = lc_message.content if hasattr(lc_message, 'content') and lc_message.content is not None else str(lc_message)
        return ChatAgentMessage(role="assistant", content=content_to_use, id=str(uuid.uuid4()))


# --- _to_langchain_base_message remains the same (it converts FROM MLflow to LangChain, which was correct previously) ---
# This function is used when converting incoming MLflow messages to LangChain messages for graph processing.
def _to_langchain_base_message(mlflow_message: ChatAgentMessage) -> BaseMessage:
    if mlflow_message.role == "user":
        return HumanMessage(content=mlflow_message.content)
    elif mlflow_message.role == "assistant":
        tool_calls = []
        if mlflow_message.tool_calls:
            # mlflow_message.tool_calls are already validated as mlflow.types.chat.ToolCall objects
            # or dicts conforming to that, so we can access tc.function.name etc.
            for tc in mlflow_message.tool_calls:
                lc_args = {}
                # Handle both dict and ToolCall object for 'tc'
                if isinstance(tc, dict): # if it's a raw dict from MLflow's side
                    if tc.get("function") and tc["function"].get("arguments"):
                        try:
                            lc_args = json.loads(tc["function"]["arguments"])
                        except json.JSONDecodeError:
                            print(f"Warning: Could not parse MLflow tool arguments as JSON: {tc['function']['arguments']}")
                            lc_args = {"raw_arguments": tc["function"]["arguments"]}
                    tool_calls.append({
                        "name": tc["function"]["name"],
                        "args": lc_args,
                        "id": tc["id"]
                    })
                else: # assuming it's mlflow.types.chat.ToolCall object if not dict
                    if tc.function and tc.function.arguments:
                        try:
                            lc_args = json.loads(tc.function.arguments)
                        except json.JSONDecodeError:
                            print(f"Warning: Could not parse MLflow tool arguments as JSON: {tc.function.arguments}")
                            lc_args = {"raw_arguments": tc.function.arguments}
                    tool_calls.append({
                        "name": tc.function.name,
                        "args": lc_args,
                        "id": tc.id
                    })

        return AIMessage(content=mlflow_message.content, tool_calls=tool_calls)
    elif mlflow_message.role == "tool":
        return ToolMessage(content=mlflow_message.content, name=mlflow_message.name, tool_call_id=mlflow_message.tool_call_id)
    elif mlflow_message.role == "system":
        return SystemMessage(content=mlflow_message.content)
    else:
        return BaseMessage(content=mlflow_message.content, type=mlflow_message.role)


# --- _parse_dict_to_langchain_message remains unchanged (it converts raw graph dicts to LangChain BaseMessages) ---
# This is crucial for the LangGraph's internal flow.
def _parse_dict_to_langchain_message(msg_dict: dict) -> BaseMessage:
    role = msg_dict.get("role")
    content = msg_dict.get("content")
    tool_calls_raw = msg_dict.get("tool_calls")
    name = msg_dict.get("name") # For tool messages

    if role == "assistant":
        lc_tool_calls = []
        if tool_calls_raw:
            for tc_raw in tool_calls_raw:
                tool_call_id = tc_raw.get("id") 
                function_info = tc_raw.get("function", {})
                
                args_dict = {}
                raw_arguments_str = function_info.get("arguments")
                if raw_arguments_str:
                    try:
                        args_dict = json.loads(raw_arguments_str)
                    except json.JSONDecodeError:
                        print(f"Warning: Could not parse tool arguments as JSON: {raw_arguments_str}")
                        args_dict = {"raw_arguments": raw_arguments_str}

                lc_tool_calls.append({
                    "name": function_info.get("name"),
                    "args": args_dict,
                    "id": tool_call_id 
                })
        return AIMessage(content=content, tool_calls=lc_tool_calls)
    elif role == "user":
        return HumanMessage(content=content)
    elif role == "tool":
        tool_call_id = msg_dict.get("tool_call_id")
        if not tool_call_id and msg_dict.get("id"): 
            tool_call_id = msg_dict.get("id")
        return ToolMessage(content=content, name=name, tool_call_id=tool_call_id)
    elif role == "system":
        return SystemMessage(content=content)
    else:
        return BaseMessage(content=str(msg_dict), type=role)


def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[ToolNode, Sequence[BaseTool]],
    system_prompt: Optional[str] = None,
) -> CompiledGraph:
    model = model.bind_tools(tools)

    def should_continue(state: ChatAgentState):
        messages = state["messages"]
        last_message = messages[-1]
        
        # Ensure that the last message is a LangChain AIMessage with tool_calls
        # LangGraph's ToolNode expects this specific type
        if isinstance(last_message, AIMessage) and last_message.tool_calls:
            return "continue"
        else:
            return "end"

    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: ([SystemMessage(content=system_prompt)] if system_prompt else [])
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    
    model_runnable = preprocessor | model

    async def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        # The model_runnable will return a LangChain BaseMessage (e.g., AIMessage)
        response = await model_runnable.ainvoke(state, config)
        # Return the LangChain message directly within the state
        return {"messages": [response]}


    workflow = StateGraph(ChatAgentState)
    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools)) 
    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent", should_continue, {"continue": "tools", "end": END}
    )
    workflow.add_edge("tools", "agent")
    return workflow.compile()


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    async def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        lc_messages = [_to_langchain_base_message(msg) for msg in messages]
        request = {"messages": lc_messages}

        response_messages = []
        print("\n--- Starting LangGraph Astream Debugging ---")
        print(f"Initial Request Messages (LangChain format): {request['messages']}")

        async for event in self.agent.astream(request, stream_mode="updates"):
            print(f"\n--- LangGraph Event Received ---")
            print(f"____________Raw Event Structure: {event}") # Keep this for verbose debugging if needed

            for node_name, node_output in event.items():
                print(f"  Node: {node_name}")
                print(f"  ********** Node Output Data: {node_output}") # Keep this for verbose debugging if needed

                if node_output and node_output.get("messages"):
                    for raw_msg in node_output["messages"]:
                        # Convert dict to LangChain BaseMessage if it's not already one
                        if isinstance(raw_msg, dict):
                            lc_msg = _parse_dict_to_langchain_message(raw_msg)
                            print(f"    Converted raw dict to LangChain Message: {lc_msg}")
                        elif isinstance(raw_msg, BaseMessage):
                            lc_msg = raw_msg
                            print(f"    Direct LangChain Message from node: {lc_msg}")
                        else:
                            print(f"    Skipping unknown message type in stream: {type(raw_msg)} - {raw_msg}")
                            continue
                        
                        # Apply your existing filtering and conversion to MLflow message
                        if isinstance(lc_msg, AIMessage):
                            print(f"      AIMessage content: '{lc_msg.content}', tool_calls: {lc_msg.tool_calls}")
                            # Only add AIMessage if it has content OR tool_calls
                            if lc_msg.content or lc_msg.tool_calls:
                                mlflow_msg = _to_mlflow_chat_agent_message(lc_msg)
                                print(f"        Converted MLflow Message for response: {mlflow_msg}")
                                for toll_call in lc_msg.tool_calls:
                                    # ToolCall(id='call_307a82b3-3568-4552-b579-02993ae0252c', type='function', function=Function(name='nimble_deep_web_search', arguments='{"query": "safest areas in Texas", "num_results": 5, "search_engine": "google_search", "parsing_type": "plain_text"}'))

                                    repsonse_test= await self.agent.ainvoke({"messages": [{"role": "user", "content": "What is the best Ethiopian restaurant on the northside of Chicago?"}]})

                                    print(f"%%%%%%%%% repsonse: {repsonse_test}")
                                    

                                response_messages.append(mlflow_msg)
                            else:
                                print("        Skipping empty AIMessage without tool calls.")
                        elif isinstance(lc_msg, HumanMessage):
                            print(f"      HumanMessage content: '{lc_msg.content}'")
                            mlflow_msg = _to_mlflow_chat_agent_message(lc_msg)
                            print(f"        Converted MLflow Message for response: {mlflow_msg}")
                            response_messages.append(mlflow_msg)
                        elif isinstance(lc_msg, ToolMessage):
                            print(f"      ToolMessage content: '{lc_msg.content}', name: {lc_msg.name}")
                            mlflow_msg = _to_mlflow_chat_agent_message(lc_msg)
                            print(f"        Converted MLflow Message for response: {mlflow_msg}")
                            response_messages.append(mlflow_msg)
                        else:
                            print(f"      Skipping non-displayable LangChain message type: {type(lc_msg)}")

        print(f"\n--- Final Assembled MLflow Response Messages: {response_messages} ---")
        return ChatAgentResponse(messages=response_messages)


    async def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        lc_messages = [_to_langchain_base_message(msg) for msg in messages]
        request = {"messages": lc_messages}

        async for event in self.agent.astream(request, stream_mode="updates"):
            for node_name, node_output in event.items():
                if node_output and node_output.get("messages"):
                    for raw_msg in node_output["messages"]:
                        if isinstance(raw_msg, dict):
                            lc_msg = _parse_dict_to_langchain_message(raw_msg)
                        elif isinstance(raw_msg, BaseMessage):
                            lc_msg = raw_msg
                        else:
                            continue # Skip unknown types

                        if isinstance(lc_msg, (AIMessage, HumanMessage, ToolMessage)):
                            if isinstance(lc_msg, AIMessage) and not lc_msg.content and not lc_msg.tool_calls:
                                continue # Skip empty AIMessages without tool calls
                            yield ChatAgentChunk(**{"delta": _to_mlflow_chat_agent_message(lc_msg)})

    # async def predict(
    #     self,
    #     messages: list[ChatAgentMessage],
    #     context: Optional[ChatContext] = None,
    #     custom_inputs: Optional[dict[str, Any]] = None,
    # ) -> ChatAgentResponse:
    #     # Convert incoming MLflow ChatAgentMessages to LangChain BaseMessages
    #     lc_messages = [_to_langchain_base_message(msg) for msg in messages]
    #     request = {"messages": lc_messages}

    #     response_messages = []
    #     async for event in self.agent.astream(request, stream_mode="updates"):
    #         for node_data in event.values():
    #             if node_data.get("messages"):
    #                 # node_data["messages"] will now contain LangChain BaseMessage types
    #                 # Convert them to MLflow ChatAgentMessage for the final response
    #                 for lc_msg in node_data["messages"]:
    #                     # Only add if it's a message type we want to return
    #                     # This avoids internal LangChain messages from appearing in the final output
    #                     if isinstance(lc_msg, (AIMessage, HumanMessage, ToolMessage)):
    #                          response_messages.append(_to_mlflow_chat_agent_message(lc_msg))
        
    #     # Filter for unique messages if the stream produces duplicates (LangGraph sometimes does)
    #     # A simple way to deduplicate if needed, though not strictly necessary if graph logic is tight
    #     # unique_response_messages = []
    #     # seen_contents = set()
    #     # for msg in response_messages:
    #     #     if msg.content not in seen_contents:
    #     #         unique_response_messages.append(msg)
    #     #         seen_contents.add(msg.content)
    #     # return ChatAgentResponse(messages=unique_response_messages)

    #     print(f"--- Final Response Messages: {response_messages} ---")
        
    #     return ChatAgentResponse(messages=response_messages)


    # async def predict_stream(
    #     self,
    #     messages: list[ChatAgentMessage],
    #     context: Optional[ChatContext] = None,
    #     custom_inputs: Optional[dict[str, Any]] = None,
    # ) -> Generator[ChatAgentChunk, None, None]:
    #     # Convert incoming MLflow ChatAgentMessages to LangChain BaseMessages
    #     lc_messages = [_to_langchain_base_message(msg) for msg in messages]
    #     request = {"messages": lc_messages}

    #     async for event in self.agent.astream(request, stream_mode="updates"):
    #         for node_data in event.values():
    #             if node_data.get("messages"):
    #                 # node_data["messages"] will contain LangChain BaseMessage types
    #                 for lc_msg in node_data["messages"]:
    #                     # Only yield if it's a message type we want to stream
    #                     if isinstance(lc_msg, (AIMessage, HumanMessage, ToolMessage)):
    #                         yield ChatAgentChunk(**{"delta": _to_mlflow_chat_agent_message(lc_msg)})

async def create_agent():
    """
    An asynchronous function to initialize and return the agent.
    This safely handles the async call to get tools.
    """
    # LLM and system prompt setup
    llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")
    
    system_prompt = ""

    # Initialize tools list
    tools = []
    uc_tool_names = ["system.ai.python_exec"]
    uc_toolkit = UCFunctionToolkit(function_names=uc_tool_names)
    tools.extend(uc_toolkit.tools)

    # Nimble MCP Tools setup
    nimble_client = MultiServerMCPClient({
        "nimble": {
            "url": "https://mcp.nimbleway.com/sse",
            "transport": "sse",
            "headers": {
                "Authorization": f"Bearer 1c7cdd787c154fe3a78ebcc3428d82c48f4fada38e0042c6a427fbe7e3b842ae"
            }
        }
    })
    
    nimble_tools = await nimble_client.get_tools()
    tools.extend(nimble_tools)

    # Create and compile the agent graph
    compiled_agent_graph = create_tool_calling_agent(llm, tools, system_prompt)
    
    # Instantiate our final agent object
    agent = LangGraphChatAgent(compiled_agent_graph)
    
    return agent


Overwriting agent.py


In [0]:
dbutils.library.restartPython()

In [0]:
# # In your Databricks notebook cell
# from mlflow.types.agent import (
#     ChatAgentChunk,
#     ChatAgentMessage,
#     ChatAgentResponse,
#     ChatContext,
# )

# # 1. Import the async factory function from your modified agent.py
# from agent import create_agent

# # 2. Await the function to get your fully initialized agent instance.
# #    Databricks notebooks support top-level await.
# AGENT = await create_agent()

# response = AGENT.predict(messages=[ChatAgentMessage(role="user", content="Can you look at Nimble MCP tool to get current weather risks in Texas?")])

# # 3. Now you can use the agent with its synchronous predict method as before.
# # response = AGENT.predict(messages=[ChatAgentMessage(role="user", content="Hello! What can you do?")])

# print(response)




# In your Databricks notebook cell
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

# 1. Import the async factory function from your modified agent.py
from agent import create_agent

# 2. Await the function to get your fully initialized agent instance.
#    Databricks notebooks support top-level await.
AGENT = await create_agent()

# 3. AWAIT THE PREDICT METHOD SINCE IT IS NOW ASYNC
# response = await AGENT.predict(messages=[ChatAgentMessage(role="user", content="Can you look at Nimble MCP tool to get current weather risks in Texas?")])


response = await AGENT.predict(messages=[ChatAgentMessage(role="user", content="What are safer areas in Texas?")])

print(response)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-10ba32fd-730a-4c03-8a2f-f5f2b474fe00/lib/python3.11/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)



--- Starting LangGraph Astream Debugging ---
Initial Request Messages (LangChain format): [HumanMessage(content='What are safer areas in Texas?', additional_kwargs={}, response_metadata={})]

--- LangGraph Event Received ---
____________Raw Event Structure: {'agent': {'messages': [{'role': 'assistant', 'content': '', 'id': 'run--805cfa78-ccb3-4202-a134-8b9e8c4fe632-0', 'tool_calls': [{'id': 'call_a762b06b-bbb7-4b69-af5d-dcb80b9272e4', 'type': 'function', 'function': {'name': 'nimble_deep_web_search', 'arguments': '{"query": "safest areas in Texas", "num_results": 5, "search_engine": "google_search", "parsing_type": "plain_text"}'}}]}]}}
  Node: agent
  ********** Node Output Data: {'messages': [{'role': 'assistant', 'content': '', 'id': 'run--805cfa78-ccb3-4202-a134-8b9e8c4fe632-0', 'tool_calls': [{'id': 'call_a762b06b-bbb7-4b69-af5d-dcb80b9272e4', 'type': 'function', 'function': {'name': 'nimble_deep_web_search', 'arguments': '{"query": "safest areas in Texas", "num_results": 5, "sea

In [0]:
print(response)


# # response = AGENT.predict(messages=[ChatAgentMessage(role="user", content="Can you look at Nimble MCP tool to get current weather risks in Texas?")])

# for event in AGENT.predict_stream(
#     {"messages": [ChatAgentMessage(role="user", content="Can you look at Nimble MCP tool to get current weather risks in Texas?")]}
# ):
#     print(event, "-----------\n")



messages=[ChatAgentMessage(role='assistant', content='', name=None, id='f976f207-dce7-4636-8ed1-4ae76ff0c5d5', tool_calls=[ToolCall(id='call_00ce73e2-cd59-4b38-9a73-f1d518649486', type='function', function=Function(name='nimble_deep_web_search', arguments='{"query": "Texas current weather risks", "num_results": 3, "search_engine": "google_search", "parsing_type": "plain_text"}'))], tool_call_id=None, attachments=None)] finish_reason=None custom_outputs=None usage=None


In [0]:
print(response)

messages=[ChatAgentMessage(role='assistant', content="Hello! I'm here to help you with various tasks and answer your questions. I can process and generate text, perform calculations, and even interact with external tools to provide more accurate and helpful responses. Here are some examples of what I can do:\n\n1. **Text-based tasks**: I can understand and respond to natural language queries, summarize content, translate text, and even generate text based on a prompt.\n2. **Problem-solving**: I can help with mathematical calculations, provide step-by-step solutions to problems, and offer explanations for complex concepts.\n3. **Information retrieval**: I can search for information on the web, retrieve data from various sources, and provide answers to factual questions.\n4. **Tool-based tasks**: I can interact with various tools to perform tasks such as data extraction, web scraping, and more.\n\nSome specific tools I can use include:\n* `nimble_targeted_engines`: Fetch lists of availab